In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import warnings
warnings.filterwarnings('ignore')

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import RobustScaler
from sklearn.neighbors import KNeighborsClassifier
from matplotlib import pyplot as plt
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from xgboost import XGBClassifier

import math

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
#for dirname, _, filenames in os.walk('C:\Projects\Jupyter\Kaggle\Forests'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

/kaggle/input/learn-together/train.csv
/kaggle/input/learn-together/sample_submission.csv
/kaggle/input/learn-together/test.csv


# Data deploying

In [2]:
data = pd.read_csv("/kaggle/input/learn-together/train.csv", sep=",")
test = pd.read_csv("/kaggle/input/learn-together/test.csv", sep=",")

In [3]:
data = data.append(test.loc[test["Id"] == 83197, :], ignore_index=True, sort = False)
data = data.append(test.loc[test["Id"] == 241544, :], ignore_index=True, sort = False)
data = data.append(test.loc[test["Id"] == 528574, :], ignore_index=True, sort = False)
data = data.append(test.loc[test["Id"] == 524376, :], ignore_index=True, sort = False)
#data = data.reset_index(drop=True)
data.loc[data["Id"] == 83197, "Cover_Type"] = 1
data.loc[data["Id"] == 241544, "Cover_Type"] = 4
data.loc[data["Id"] == 528574, "Cover_Type"] = 1
data.loc[data["Id"] == 524376, "Cover_Type"] = 1
data["Cover_Type"] = data["Cover_Type"].astype(np.int64)
X = data.drop(["Cover_Type"], axis=1)
y = pd.DataFrame(data, columns=["Cover_Type"])

In [4]:
X.drop(['Id'], axis=1, inplace=True)
#X.drop(['Id', 'Soil_Type7', 'Soil_Type15'], axis=1, inplace=True)
test_dr = test.copy()
#test_dr = test.drop(['Soil_Type7', 'Soil_Type15'], axis=1)

In [5]:
WA1ind = data.loc[data['Wilderness_Area1'] == 1].index
WA2ind = data.loc[data['Wilderness_Area2'] == 1].index
WA3ind = data.loc[data['Wilderness_Area3'] == 1].index
WA4ind = data.loc[data['Wilderness_Area4'] == 1].index

In [6]:
forest_1 = X.loc[X['Wilderness_Area1'] == 1].copy()
test_forest_1 = test_dr.loc[test_dr['Wilderness_Area1'] == 1].copy()
y_forest_1 = y.iloc[WA1ind].copy()
forest_2 = X.loc[X['Wilderness_Area2'] == 1].copy()
test_forest_2 = test_dr.loc[test_dr['Wilderness_Area2'] == 1].copy()
y_forest_2 = y.iloc[WA2ind].copy()
forest_3 = X.loc[X['Wilderness_Area3'] == 1].copy()
test_forest_3 = test_dr.loc[test_dr['Wilderness_Area3'] == 1].copy()
y_forest_3 = y.iloc[WA3ind].copy()
forest_4 = X.loc[X['Wilderness_Area4'] == 1].copy()
test_forest_4 = test_dr.loc[test_dr['Wilderness_Area4'] == 1].copy()
y_forest_4 = y.iloc[WA4ind].copy()

In [7]:
def feature_remover(sc_data):
    drop_list = []
    for ind in range(0, sc_data.shape[1]):
        if sc_data.iloc[:,ind].nunique() == 1:
            #print("Drop {}".format(sc_data.keys()[ind]))
            drop_list.append(ind)
    sc_data.drop(sc_data.keys()[drop_list], axis=1, inplace = True)
    return sc_data

In [8]:
all_forest_list = [forest_1, forest_2, forest_3, forest_4, test_forest_1, test_forest_2, test_forest_3, test_forest_4]
for fr in all_forest_list:
    fr = feature_remover(fr)
    print(fr.nunique())

Elevation                              905
Aspect                                 355
Slope                                   47
Horizontal_Distance_To_Hydrology       237
Vertical_Distance_To_Hydrology         305
Horizontal_Distance_To_Roadways       2109
Hillshade_9am                          132
Hillshade_Noon                         123
Hillshade_3pm                          219
Horizontal_Distance_To_Fire_Points    2020
Soil_Type7                               2
Soil_Type8                               2
Soil_Type9                               2
Soil_Type12                              2
Soil_Type16                              2
Soil_Type18                              2
Soil_Type19                              2
Soil_Type20                              2
Soil_Type22                              2
Soil_Type23                              2
Soil_Type24                              2
Soil_Type29                              2
Soil_Type30                              2
Soil_Type35

In [9]:
res_1 = test_forest_1[['Id']].copy()
res_2 = test_forest_2[['Id']].copy()
res_3 = test_forest_3[['Id']].copy()
res_4 = test_forest_4[['Id']].copy()
res_1 = res_1.reset_index(drop=True)
res_2 = res_2.reset_index(drop=True)
res_3 = res_3.reset_index(drop=True)
res_4 = res_4.reset_index(drop=True)

In [10]:
test_forest_1.drop(["Id"], axis=1, inplace = True)
test_forest_2.drop(["Id"], axis=1, inplace = True)
test_forest_3.drop(["Id"], axis=1, inplace = True)
test_forest_4.drop(["Id"], axis=1, inplace = True)

In [11]:
fr_1 = pd.concat([forest_1, y_forest_1], axis=1)
fr_2 = pd.concat([forest_2, y_forest_2], axis=1)
fr_3 = pd.concat([forest_3, y_forest_3], axis=1)
fr_4 = pd.concat([forest_4, y_forest_4], axis=1)

In [12]:
forest_list = [forest_1, forest_2, forest_3, forest_4]
y_forest_list = [y_forest_1, y_forest_2, y_forest_3, y_forest_4]
test_list = [test_forest_1, test_forest_2, test_forest_3, test_forest_4]
res_ind_list = [res_1, res_2, res_3, res_4]
fr_list = [fr_1, fr_2, fr_3, fr_4]

# Features creation

### Water below

In [13]:
#for ind in range(0, 4):
#    forest_list[ind]["Water_below"] = 1
#    forest_list[ind].loc[forest_list[ind]["Vertical_Distance_To_Hydrology"] < 0, "Water_below"] = 0
#    test_list[ind]["Water_below"] = 1
#    test_list[ind].loc[test_list[ind]["Vertical_Distance_To_Hydrology"] < 0, "Water_below"] = 0
#    fr_list[ind]["Water_below"] = 1
#    fr_list[ind].loc[fr_list[ind]["Vertical_Distance_To_Hydrology"] < 0, "Water_below"] = 0

### Water distance

In [14]:
#for ind in range(0, 4):
#    forest_list[ind]["Water_distance"] = (forest_list[ind]["Vertical_Distance_To_Hydrology"] ** 2 + forest_list[ind]["Horizontal_Distance_To_Hydrology"] ** 2) ** 0.5
#    test_list[ind]["Water_distance"] = (test_list[ind]["Vertical_Distance_To_Hydrology"] ** 2 + test_list[ind]["Horizontal_Distance_To_Hydrology"] ** 2) ** 0.5
#    fr_list[ind]["Water_distance"] = (fr_list[ind]["Vertical_Distance_To_Hydrology"] ** 2 + fr_list[ind]["Horizontal_Distance_To_Hydrology"] ** 2) ** 0.5

### Type probability

In [15]:

forest_1["P_Cover_Type_1"] = 0
forest_1["P_Cover_Type_2"] = 0
forest_1["P_Cover_Type_5"] = 0
forest_1["P_Cover_Type_7"] = 0
test_forest_1["P_Cover_Type_1"] = 0
test_forest_1["P_Cover_Type_2"] = 0
test_forest_1["P_Cover_Type_5"] = 0
test_forest_1["P_Cover_Type_7"] = 0

forest_2["P_Cover_Type_1"] = 0
forest_2["P_Cover_Type_2"] = 0
forest_2["P_Cover_Type_7"] = 0
test_forest_2["P_Cover_Type_1"] = 0
test_forest_2["P_Cover_Type_2"] = 0
test_forest_2["P_Cover_Type_7"] = 0

forest_3["P_Cover_Type_1"] = 0
forest_3["P_Cover_Type_2"] = 0
forest_3["P_Cover_Type_3"] = 0
forest_3["P_Cover_Type_5"] = 0
forest_3["P_Cover_Type_6"] = 0
forest_3["P_Cover_Type_7"] = 0
test_forest_3["P_Cover_Type_1"] = 0
test_forest_3["P_Cover_Type_2"] = 0
test_forest_3["P_Cover_Type_3"] = 0
test_forest_3["P_Cover_Type_5"] = 0
test_forest_3["P_Cover_Type_6"] = 0
test_forest_3["P_Cover_Type_7"] = 0

forest_4["P_Cover_Type_2"] = 0
forest_4["P_Cover_Type_3"] = 0
forest_4["P_Cover_Type_4"] = 0
forest_4["P_Cover_Type_6"] = 0
test_forest_4["P_Cover_Type_2"] = 0
test_forest_4["P_Cover_Type_3"] = 0
test_forest_4["P_Cover_Type_4"] = 0
test_forest_4["P_Cover_Type_6"] = 0


In [16]:

forest_1.loc[forest_1["Soil_Type7"] == 1, "P_Cover_Type_2"] = 1
forest_1.loc[forest_1["Soil_Type8"] == 1, "P_Cover_Type_2"] = 1
forest_1.loc[forest_1["Soil_Type9"] == 1, "P_Cover_Type_1"] = 0.1
forest_1.loc[forest_1["Soil_Type9"] == 1, "P_Cover_Type_2"] = 0.9
forest_1.loc[forest_1["Soil_Type12"] == 1, "P_Cover_Type_1"] = 0.1
forest_1.loc[forest_1["Soil_Type12"] == 1, "P_Cover_Type_2"] = 0.9
forest_1.loc[forest_1["Soil_Type16"] == 1, "P_Cover_Type_1"] = 0.3
forest_1.loc[forest_1["Soil_Type16"] == 1, "P_Cover_Type_2"] = 0.4
forest_1.loc[forest_1["Soil_Type16"] == 1, "P_Cover_Type_5"] = 0.3
forest_1.loc[forest_1["Soil_Type18"] == 1, "P_Cover_Type_2"] = 0.27
forest_1.loc[forest_1["Soil_Type18"] == 1, "P_Cover_Type_5"] = 0.73
forest_1.loc[forest_1["Soil_Type19"] == 1, "P_Cover_Type_1"] = 0.43
forest_1.loc[forest_1["Soil_Type19"] == 1, "P_Cover_Type_2"] = 0.57
forest_1.loc[forest_1["Soil_Type20"] == 1, "P_Cover_Type_1"] = 0.35
forest_1.loc[forest_1["Soil_Type20"] == 1, "P_Cover_Type_2"] = 0.65
forest_1.loc[forest_1["Soil_Type22"] == 1, "P_Cover_Type_1"] = 0.81
forest_1.loc[forest_1["Soil_Type22"] == 1, "P_Cover_Type_2"] = 0.19
forest_1.loc[forest_1["Soil_Type23"] == 1, "P_Cover_Type_1"] = 0.54
forest_1.loc[forest_1["Soil_Type23"] == 1, "P_Cover_Type_2"] = 0.21
forest_1.loc[forest_1["Soil_Type23"] == 1, "P_Cover_Type_5"] = 0.16
forest_1.loc[forest_1["Soil_Type23"] == 1, "P_Cover_Type_7"] = 0.09
forest_1.loc[forest_1["Soil_Type24"] == 1, "P_Cover_Type_1"] = 0.24
forest_1.loc[forest_1["Soil_Type24"] == 1, "P_Cover_Type_2"] = 0.5
forest_1.loc[forest_1["Soil_Type24"] == 1, "P_Cover_Type_5"] = 0.26
forest_1.loc[forest_1["Soil_Type29"] == 1, "P_Cover_Type_1"] = 0.31
forest_1.loc[forest_1["Soil_Type29"] == 1, "P_Cover_Type_2"] = 0.43
forest_1.loc[forest_1["Soil_Type29"] == 1, "P_Cover_Type_5"] = 0.2
forest_1.loc[forest_1["Soil_Type29"] == 1, "P_Cover_Type_7"] = 0.06
forest_1.loc[forest_1["Soil_Type30"] == 1, "P_Cover_Type_1"] = 0.11
forest_1.loc[forest_1["Soil_Type30"] == 1, "P_Cover_Type_2"] = 0.2
forest_1.loc[forest_1["Soil_Type30"] == 1, "P_Cover_Type_5"] = 0.66
forest_1.loc[forest_1["Soil_Type30"] == 1, "P_Cover_Type_7"] = 0.03
forest_1.loc[forest_1["Soil_Type35"] == 1, "P_Cover_Type_1"] = 0.04
forest_1.loc[forest_1["Soil_Type35"] == 1, "P_Cover_Type_7"] = 0.96
forest_1.loc[forest_1["Soil_Type37"] == 1, "P_Cover_Type_7"] = 1
forest_1.loc[forest_1["Soil_Type38"] == 1, "P_Cover_Type_1"] = 0.21
forest_1.loc[forest_1["Soil_Type38"] == 1, "P_Cover_Type_2"] = 0.01
forest_1.loc[forest_1["Soil_Type38"] == 1, "P_Cover_Type_7"] = 0.78
forest_1.loc[forest_1["Soil_Type39"] == 1, "P_Cover_Type_1"] = 0.31
forest_1.loc[forest_1["Soil_Type39"] == 1, "P_Cover_Type_7"] = 0.69
forest_1.loc[forest_1["Soil_Type40"] == 1, "P_Cover_Type_1"] = 0.4
forest_1.loc[forest_1["Soil_Type40"] == 1, "P_Cover_Type_7"] = 0.6

test_forest_1.loc[test_forest_1["Soil_Type7"] == 1, "P_Cover_Type_2"] = 1
test_forest_1.loc[test_forest_1["Soil_Type8"] == 1, "P_Cover_Type_2"] = 1
test_forest_1.loc[test_forest_1["Soil_Type9"] == 1, "P_Cover_Type_1"] = 0.1
test_forest_1.loc[test_forest_1["Soil_Type9"] == 1, "P_Cover_Type_2"] = 0.9
test_forest_1.loc[test_forest_1["Soil_Type12"] == 1, "P_Cover_Type_1"] = 0.1
test_forest_1.loc[test_forest_1["Soil_Type12"] == 1, "P_Cover_Type_2"] = 0.9
test_forest_1.loc[test_forest_1["Soil_Type16"] == 1, "P_Cover_Type_1"] = 0.3
test_forest_1.loc[test_forest_1["Soil_Type16"] == 1, "P_Cover_Type_2"] = 0.4
test_forest_1.loc[test_forest_1["Soil_Type16"] == 1, "P_Cover_Type_5"] = 0.3
test_forest_1.loc[test_forest_1["Soil_Type18"] == 1, "P_Cover_Type_2"] = 0.27
test_forest_1.loc[test_forest_1["Soil_Type18"] == 1, "P_Cover_Type_5"] = 0.73
test_forest_1.loc[test_forest_1["Soil_Type19"] == 1, "P_Cover_Type_1"] = 0.43
test_forest_1.loc[test_forest_1["Soil_Type19"] == 1, "P_Cover_Type_2"] = 0.57
test_forest_1.loc[test_forest_1["Soil_Type20"] == 1, "P_Cover_Type_1"] = 0.35
test_forest_1.loc[test_forest_1["Soil_Type20"] == 1, "P_Cover_Type_2"] = 0.65
test_forest_1.loc[test_forest_1["Soil_Type22"] == 1, "P_Cover_Type_1"] = 0.81
test_forest_1.loc[test_forest_1["Soil_Type22"] == 1, "P_Cover_Type_2"] = 0.19
test_forest_1.loc[test_forest_1["Soil_Type23"] == 1, "P_Cover_Type_1"] = 0.54
test_forest_1.loc[test_forest_1["Soil_Type23"] == 1, "P_Cover_Type_2"] = 0.21
test_forest_1.loc[test_forest_1["Soil_Type23"] == 1, "P_Cover_Type_5"] = 0.16
test_forest_1.loc[test_forest_1["Soil_Type23"] == 1, "P_Cover_Type_7"] = 0.09
test_forest_1.loc[test_forest_1["Soil_Type24"] == 1, "P_Cover_Type_1"] = 0.24
test_forest_1.loc[test_forest_1["Soil_Type24"] == 1, "P_Cover_Type_2"] = 0.5
test_forest_1.loc[test_forest_1["Soil_Type24"] == 1, "P_Cover_Type_5"] = 0.26
test_forest_1.loc[test_forest_1["Soil_Type29"] == 1, "P_Cover_Type_1"] = 0.31
test_forest_1.loc[test_forest_1["Soil_Type29"] == 1, "P_Cover_Type_2"] = 0.43
test_forest_1.loc[test_forest_1["Soil_Type29"] == 1, "P_Cover_Type_5"] = 0.2
test_forest_1.loc[test_forest_1["Soil_Type29"] == 1, "P_Cover_Type_7"] = 0.06
test_forest_1.loc[test_forest_1["Soil_Type30"] == 1, "P_Cover_Type_1"] = 0.11
test_forest_1.loc[test_forest_1["Soil_Type30"] == 1, "P_Cover_Type_2"] = 0.2
test_forest_1.loc[test_forest_1["Soil_Type30"] == 1, "P_Cover_Type_5"] = 0.66
test_forest_1.loc[test_forest_1["Soil_Type30"] == 1, "P_Cover_Type_7"] = 0.03
test_forest_1.loc[test_forest_1["Soil_Type35"] == 1, "P_Cover_Type_1"] = 0.04
test_forest_1.loc[test_forest_1["Soil_Type35"] == 1, "P_Cover_Type_7"] = 0.96
test_forest_1.loc[test_forest_1["Soil_Type37"] == 1, "P_Cover_Type_7"] = 1
test_forest_1.loc[test_forest_1["Soil_Type38"] == 1, "P_Cover_Type_1"] = 0.21
test_forest_1.loc[test_forest_1["Soil_Type38"] == 1, "P_Cover_Type_2"] = 0.01
test_forest_1.loc[test_forest_1["Soil_Type38"] == 1, "P_Cover_Type_7"] = 0.78
test_forest_1.loc[test_forest_1["Soil_Type39"] == 1, "P_Cover_Type_1"] = 0.31
test_forest_1.loc[test_forest_1["Soil_Type39"] == 1, "P_Cover_Type_7"] = 0.69
test_forest_1.loc[test_forest_1["Soil_Type40"] == 1, "P_Cover_Type_1"] = 0.4
test_forest_1.loc[test_forest_1["Soil_Type40"] == 1, "P_Cover_Type_7"] = 0.6

forest_2.loc[forest_2["Soil_Type13"] == 1, "P_Cover_Type_1"] = 1
forest_2.loc[forest_2["Soil_Type16"] == 1, "P_Cover_Type_1"] = 1
forest_2.loc[forest_2["Soil_Type18"] == 1, "P_Cover_Type_1"] = 1
forest_2.loc[forest_2["Soil_Type19"] == 1, "P_Cover_Type_1"] = 1
forest_2.loc[forest_2["Soil_Type20"] == 1, "P_Cover_Type_1"] = 1
forest_2.loc[forest_2["Soil_Type22"] == 1, "P_Cover_Type_1"] = 0.82
forest_2.loc[forest_2["Soil_Type22"] == 1, "P_Cover_Type_2"] = 0.18
forest_2.loc[forest_2["Soil_Type23"] == 1, "P_Cover_Type_1"] = 0.69
forest_2.loc[forest_2["Soil_Type23"] == 1, "P_Cover_Type_2"] = 0.31
forest_2.loc[forest_2["Soil_Type24"] == 1, "P_Cover_Type_1"] = 0.28
forest_2.loc[forest_2["Soil_Type24"] == 1, "P_Cover_Type_2"] = 0.23
forest_2.loc[forest_2["Soil_Type24"] == 1, "P_Cover_Type_7"] = 0.49
forest_2.loc[forest_2["Soil_Type25"] == 1, "P_Cover_Type_2"] = 1
forest_2.loc[forest_2["Soil_Type29"] == 1, "P_Cover_Type_1"] = 1
forest_2.loc[forest_2["Soil_Type31"] == 1, "P_Cover_Type_1"] = 1
forest_2.loc[forest_2["Soil_Type32"] == 1, "P_Cover_Type_1"] = 0.61
forest_2.loc[forest_2["Soil_Type32"] == 1, "P_Cover_Type_2"] = 0.39
forest_2.loc[forest_2["Soil_Type33"] == 1, "P_Cover_Type_1"] = 0.7
forest_2.loc[forest_2["Soil_Type33"] == 1, "P_Cover_Type_2"] = 0.18
forest_2.loc[forest_2["Soil_Type33"] == 1, "P_Cover_Type_7"] = 0.12
forest_2.loc[forest_2["Soil_Type35"] == 1, "P_Cover_Type_1"] = 0.33
forest_2.loc[forest_2["Soil_Type35"] == 1, "P_Cover_Type_2"] = 0.17
forest_2.loc[forest_2["Soil_Type35"] == 1, "P_Cover_Type_7"] = 0.5
forest_2.loc[forest_2["Soil_Type38"] == 1, "P_Cover_Type_1"] = 0.16
forest_2.loc[forest_2["Soil_Type38"] == 1, "P_Cover_Type_2"] = 0.07
forest_2.loc[forest_2["Soil_Type38"] == 1, "P_Cover_Type_7"] = 0.77
forest_2.loc[forest_2["Soil_Type39"] == 1, "P_Cover_Type_1"] = 0.08
forest_2.loc[forest_2["Soil_Type39"] == 1, "P_Cover_Type_7"] = 0.92
forest_2.loc[forest_2["Soil_Type40"] == 1, "P_Cover_Type_1"] = 0.16
forest_2.loc[forest_2["Soil_Type40"] == 1, "P_Cover_Type_2"] = 0.07
forest_2.loc[forest_2["Soil_Type40"] == 1, "P_Cover_Type_7"] = 0.77

test_forest_2.loc[test_forest_2["Soil_Type13"] == 1, "P_Cover_Type_1"] = 1
test_forest_2.loc[test_forest_2["Soil_Type16"] == 1, "P_Cover_Type_1"] = 1
test_forest_2.loc[test_forest_2["Soil_Type18"] == 1, "P_Cover_Type_1"] = 1
test_forest_2.loc[test_forest_2["Soil_Type19"] == 1, "P_Cover_Type_1"] = 1
test_forest_2.loc[test_forest_2["Soil_Type20"] == 1, "P_Cover_Type_1"] = 1
test_forest_2.loc[test_forest_2["Soil_Type22"] == 1, "P_Cover_Type_1"] = 0.82
test_forest_2.loc[test_forest_2["Soil_Type22"] == 1, "P_Cover_Type_2"] = 0.18
test_forest_2.loc[test_forest_2["Soil_Type23"] == 1, "P_Cover_Type_1"] = 0.69
test_forest_2.loc[test_forest_2["Soil_Type23"] == 1, "P_Cover_Type_2"] = 0.31
test_forest_2.loc[test_forest_2["Soil_Type24"] == 1, "P_Cover_Type_1"] = 0.28
test_forest_2.loc[test_forest_2["Soil_Type24"] == 1, "P_Cover_Type_2"] = 0.23
test_forest_2.loc[test_forest_2["Soil_Type24"] == 1, "P_Cover_Type_7"] = 0.49
test_forest_2.loc[test_forest_2["Soil_Type25"] == 1, "P_Cover_Type_2"] = 1
test_forest_2.loc[test_forest_2["Soil_Type29"] == 1, "P_Cover_Type_1"] = 1
test_forest_2.loc[test_forest_2["Soil_Type31"] == 1, "P_Cover_Type_1"] = 1
test_forest_2.loc[test_forest_2["Soil_Type32"] == 1, "P_Cover_Type_1"] = 0.61
test_forest_2.loc[test_forest_2["Soil_Type32"] == 1, "P_Cover_Type_2"] = 0.39
test_forest_2.loc[test_forest_2["Soil_Type33"] == 1, "P_Cover_Type_1"] = 0.7
test_forest_2.loc[test_forest_2["Soil_Type33"] == 1, "P_Cover_Type_2"] = 0.18
test_forest_2.loc[test_forest_2["Soil_Type33"] == 1, "P_Cover_Type_7"] = 0.12
test_forest_2.loc[test_forest_2["Soil_Type35"] == 1, "P_Cover_Type_1"] = 0.33
test_forest_2.loc[test_forest_2["Soil_Type35"] == 1, "P_Cover_Type_2"] = 0.17
test_forest_2.loc[test_forest_2["Soil_Type35"] == 1, "P_Cover_Type_7"] = 0.5
test_forest_2.loc[test_forest_2["Soil_Type38"] == 1, "P_Cover_Type_1"] = 0.16
test_forest_2.loc[test_forest_2["Soil_Type38"] == 1, "P_Cover_Type_2"] = 0.07
test_forest_2.loc[test_forest_2["Soil_Type38"] == 1, "P_Cover_Type_7"] = 0.77
test_forest_2.loc[test_forest_2["Soil_Type39"] == 1, "P_Cover_Type_1"] = 0.08
test_forest_2.loc[test_forest_2["Soil_Type39"] == 1, "P_Cover_Type_7"] = 0.92
test_forest_2.loc[test_forest_2["Soil_Type40"] == 1, "P_Cover_Type_1"] = 0.16
test_forest_2.loc[test_forest_2["Soil_Type40"] == 1, "P_Cover_Type_2"] = 0.07
test_forest_2.loc[test_forest_2["Soil_Type40"] == 1, "P_Cover_Type_7"] = 0.77

forest_3.loc[forest_3["Soil_Type2"] == 1, "P_Cover_Type_2"] = 0.01
forest_3.loc[forest_3["Soil_Type2"] == 1, "P_Cover_Type_3"] = 0.57
forest_3.loc[forest_3["Soil_Type2"] == 1, "P_Cover_Type_5"] = 0.18
forest_3.loc[forest_3["Soil_Type2"] == 1, "P_Cover_Type_6"] = 0.24
forest_3.loc[forest_3["Soil_Type3"] == 1, "P_Cover_Type_2"] = 0.16
forest_3.loc[forest_3["Soil_Type3"] == 1, "P_Cover_Type_3"] = 0.7
forest_3.loc[forest_3["Soil_Type3"] == 1, "P_Cover_Type_6"] = 0.14
forest_3.loc[forest_3["Soil_Type4"] == 1, "P_Cover_Type_1"] = 0.01
forest_3.loc[forest_3["Soil_Type4"] == 1, "P_Cover_Type_2"] = 0.03
forest_3.loc[forest_3["Soil_Type4"] == 1, "P_Cover_Type_3"] = 0.64
forest_3.loc[forest_3["Soil_Type4"] == 1, "P_Cover_Type_5"] = 0.2
forest_3.loc[forest_3["Soil_Type4"] == 1, "P_Cover_Type_6"] = 0.11
forest_3.loc[forest_3["Soil_Type4"] == 1, "P_Cover_Type_7"] = 0.01
forest_3.loc[forest_3["Soil_Type10"] == 1, "P_Cover_Type_1"] = 0.01
forest_3.loc[forest_3["Soil_Type10"] == 1, "P_Cover_Type_2"] = 0.1
forest_3.loc[forest_3["Soil_Type10"] == 1, "P_Cover_Type_3"] = 0.15
forest_3.loc[forest_3["Soil_Type10"] == 1, "P_Cover_Type_5"] = 0.1
forest_3.loc[forest_3["Soil_Type10"] == 1, "P_Cover_Type_6"] = 0.63
forest_3.loc[forest_3["Soil_Type11"] == 1, "P_Cover_Type_1"] = 0.01
forest_3.loc[forest_3["Soil_Type11"] == 1, "P_Cover_Type_2"] = 0.2
forest_3.loc[forest_3["Soil_Type11"] == 1, "P_Cover_Type_3"] = 0.23
forest_3.loc[forest_3["Soil_Type11"] == 1, "P_Cover_Type_5"] = 0.46
forest_3.loc[forest_3["Soil_Type11"] == 1, "P_Cover_Type_6"] = 0.1
forest_3.loc[forest_3["Soil_Type13"] == 1, "P_Cover_Type_1"] = 0.03
forest_3.loc[forest_3["Soil_Type13"] == 1, "P_Cover_Type_2"] = 0.18
forest_3.loc[forest_3["Soil_Type13"] == 1, "P_Cover_Type_3"] = 0.01
forest_3.loc[forest_3["Soil_Type13"] == 1, "P_Cover_Type_5"] = 0.64
forest_3.loc[forest_3["Soil_Type13"] == 1, "P_Cover_Type_6"] = 0.14
forest_3.loc[forest_3["Soil_Type14"] == 1, "P_Cover_Type_3"] = 0.06
forest_3.loc[forest_3["Soil_Type14"] == 1, "P_Cover_Type_6"] = 0.94
forest_3.loc[forest_3["Soil_Type16"] == 1, "P_Cover_Type_2"] = 0.07
forest_3.loc[forest_3["Soil_Type16"] == 1, "P_Cover_Type_3"] = 0.07
forest_3.loc[forest_3["Soil_Type16"] == 1, "P_Cover_Type_6"] = 0.86
forest_3.loc[forest_3["Soil_Type17"] == 1, "P_Cover_Type_1"] = 0.01
forest_3.loc[forest_3["Soil_Type17"] == 1, "P_Cover_Type_2"] = 0.03
forest_3.loc[forest_3["Soil_Type17"] == 1, "P_Cover_Type_3"] = 0.05
forest_3.loc[forest_3["Soil_Type17"] == 1, "P_Cover_Type_5"] = 0.57
forest_3.loc[forest_3["Soil_Type17"] == 1, "P_Cover_Type_6"] = 0.34
forest_3.loc[forest_3["Soil_Type19"] == 1, "P_Cover_Type_1"] = 0.14
forest_3.loc[forest_3["Soil_Type19"] == 1, "P_Cover_Type_2"] = 0.05
forest_3.loc[forest_3["Soil_Type19"] == 1, "P_Cover_Type_5"] = 0.81
forest_3.loc[forest_3["Soil_Type20"] == 1, "P_Cover_Type_1"] = 0.21
forest_3.loc[forest_3["Soil_Type20"] == 1, "P_Cover_Type_2"] = 0.1
forest_3.loc[forest_3["Soil_Type20"] == 1, "P_Cover_Type_5"] = 0.1
forest_3.loc[forest_3["Soil_Type20"] == 1, "P_Cover_Type_6"] = 0.59
forest_3.loc[forest_3["Soil_Type21"] == 1, "P_Cover_Type_1"] = 0.75
forest_3.loc[forest_3["Soil_Type21"] == 1, "P_Cover_Type_7"] = 0.25
forest_3.loc[forest_3["Soil_Type22"] == 1, "P_Cover_Type_1"] = 0.76
forest_3.loc[forest_3["Soil_Type22"] == 1, "P_Cover_Type_2"] = 0.07
forest_3.loc[forest_3["Soil_Type22"] == 1, "P_Cover_Type_7"] = 0.17
forest_3.loc[forest_3["Soil_Type23"] == 1, "P_Cover_Type_1"] = 0.41
forest_3.loc[forest_3["Soil_Type23"] == 1, "P_Cover_Type_2"] = 0.17
forest_3.loc[forest_3["Soil_Type23"] == 1, "P_Cover_Type_5"] = 0.29
forest_3.loc[forest_3["Soil_Type23"] == 1, "P_Cover_Type_6"] = 0.01
forest_3.loc[forest_3["Soil_Type23"] == 1, "P_Cover_Type_7"] = 0.12
forest_3.loc[forest_3["Soil_Type24"] == 1, "P_Cover_Type_1"] = 0.61
forest_3.loc[forest_3["Soil_Type24"] == 1, "P_Cover_Type_2"] = 0.24
forest_3.loc[forest_3["Soil_Type24"] == 1, "P_Cover_Type_6"] = 0.08
forest_3.loc[forest_3["Soil_Type24"] == 1, "P_Cover_Type_7"] = 0.07
forest_3.loc[forest_3["Soil_Type26"] == 1, "P_Cover_Type_1"] = 0.13
forest_3.loc[forest_3["Soil_Type26"] == 1, "P_Cover_Type_2"] = 0.35
forest_3.loc[forest_3["Soil_Type26"] == 1, "P_Cover_Type_5"] = 0.52
forest_3.loc[forest_3["Soil_Type27"] == 1, "P_Cover_Type_1"] = 0.47
forest_3.loc[forest_3["Soil_Type27"] == 1, "P_Cover_Type_2"] = 0.33
forest_3.loc[forest_3["Soil_Type27"] == 1, "P_Cover_Type_7"] = 0.2
forest_3.loc[forest_3["Soil_Type28"] == 1, "P_Cover_Type_2"] = 0.67
forest_3.loc[forest_3["Soil_Type28"] == 1, "P_Cover_Type_7"] = 0.33
forest_3.loc[forest_3["Soil_Type31"] == 1, "P_Cover_Type_1"] = 0.33
forest_3.loc[forest_3["Soil_Type31"] == 1, "P_Cover_Type_2"] = 0.3
forest_3.loc[forest_3["Soil_Type31"] == 1, "P_Cover_Type_5"] = 0.27
forest_3.loc[forest_3["Soil_Type31"] == 1, "P_Cover_Type_6"] = 0.02
forest_3.loc[forest_3["Soil_Type31"] == 1, "P_Cover_Type_7"] = 0.08
forest_3.loc[forest_3["Soil_Type32"] == 1, "P_Cover_Type_1"] = 0.32
forest_3.loc[forest_3["Soil_Type32"] == 1, "P_Cover_Type_2"] = 0.37
forest_3.loc[forest_3["Soil_Type32"] == 1, "P_Cover_Type_3"] = 0.01
forest_3.loc[forest_3["Soil_Type32"] == 1, "P_Cover_Type_5"] = 0.15
forest_3.loc[forest_3["Soil_Type32"] == 1, "P_Cover_Type_6"] = 0.05
forest_3.loc[forest_3["Soil_Type32"] == 1, "P_Cover_Type_7"] = 0.1
forest_3.loc[forest_3["Soil_Type33"] == 1, "P_Cover_Type_1"] = 0.27
forest_3.loc[forest_3["Soil_Type33"] == 1, "P_Cover_Type_2"] = 0.31
forest_3.loc[forest_3["Soil_Type33"] == 1, "P_Cover_Type_5"] = 0.2
forest_3.loc[forest_3["Soil_Type33"] == 1, "P_Cover_Type_6"] = 0.11
forest_3.loc[forest_3["Soil_Type33"] == 1, "P_Cover_Type_7"] = 0.11
forest_3.loc[forest_3["Soil_Type34"] == 1, "P_Cover_Type_2"] = 0.55
forest_3.loc[forest_3["Soil_Type34"] == 1, "P_Cover_Type_5"] = 0.18
forest_3.loc[forest_3["Soil_Type34"] == 1, "P_Cover_Type_6"] = 0.09
forest_3.loc[forest_3["Soil_Type34"] == 1, "P_Cover_Type_7"] = 0.18
forest_3.loc[forest_3["Soil_Type35"] == 1, "P_Cover_Type_7"] = 1
forest_3.loc[forest_3["Soil_Type36"] == 1, "P_Cover_Type_2"] = 0.2
forest_3.loc[forest_3["Soil_Type36"] == 1, "P_Cover_Type_7"] = 0.8
forest_3.loc[forest_3["Soil_Type37"] == 1, "P_Cover_Type_7"] = 1
forest_3.loc[forest_3["Soil_Type38"] == 1, "P_Cover_Type_1"] = 0.05
forest_3.loc[forest_3["Soil_Type38"] == 1, "P_Cover_Type_7"] = 0.95
forest_3.loc[forest_3["Soil_Type39"] == 1, "P_Cover_Type_1"] = 0.04
forest_3.loc[forest_3["Soil_Type39"] == 1, "P_Cover_Type_2"] = 0.01
forest_3.loc[forest_3["Soil_Type39"] == 1, "P_Cover_Type_7"] = 0.95
forest_3.loc[forest_3["Soil_Type40"] == 1, "P_Cover_Type_1"] = 0.01
forest_3.loc[forest_3["Soil_Type40"] == 1, "P_Cover_Type_2"] = 0.01
forest_3.loc[forest_3["Soil_Type40"] == 1, "P_Cover_Type_7"] = 0.98

test_forest_3.loc[test_forest_3["Soil_Type2"] == 1, "P_Cover_Type_2"] = 0.01
test_forest_3.loc[test_forest_3["Soil_Type2"] == 1, "P_Cover_Type_3"] = 0.57
test_forest_3.loc[test_forest_3["Soil_Type2"] == 1, "P_Cover_Type_5"] = 0.18
test_forest_3.loc[test_forest_3["Soil_Type2"] == 1, "P_Cover_Type_6"] = 0.24
test_forest_3.loc[test_forest_3["Soil_Type3"] == 1, "P_Cover_Type_2"] = 0.16
test_forest_3.loc[test_forest_3["Soil_Type3"] == 1, "P_Cover_Type_3"] = 0.7
test_forest_3.loc[test_forest_3["Soil_Type3"] == 1, "P_Cover_Type_6"] = 0.14
test_forest_3.loc[test_forest_3["Soil_Type4"] == 1, "P_Cover_Type_1"] = 0.01
test_forest_3.loc[test_forest_3["Soil_Type4"] == 1, "P_Cover_Type_2"] = 0.03
test_forest_3.loc[test_forest_3["Soil_Type4"] == 1, "P_Cover_Type_3"] = 0.64
test_forest_3.loc[test_forest_3["Soil_Type4"] == 1, "P_Cover_Type_5"] = 0.2
test_forest_3.loc[test_forest_3["Soil_Type4"] == 1, "P_Cover_Type_6"] = 0.11
test_forest_3.loc[test_forest_3["Soil_Type4"] == 1, "P_Cover_Type_7"] = 0.01
test_forest_3.loc[test_forest_3["Soil_Type10"] == 1, "P_Cover_Type_1"] = 0.01
test_forest_3.loc[test_forest_3["Soil_Type10"] == 1, "P_Cover_Type_2"] = 0.1
test_forest_3.loc[test_forest_3["Soil_Type10"] == 1, "P_Cover_Type_3"] = 0.15
test_forest_3.loc[test_forest_3["Soil_Type10"] == 1, "P_Cover_Type_5"] = 0.1
test_forest_3.loc[test_forest_3["Soil_Type10"] == 1, "P_Cover_Type_6"] = 0.63
test_forest_3.loc[test_forest_3["Soil_Type11"] == 1, "P_Cover_Type_1"] = 0.01
test_forest_3.loc[test_forest_3["Soil_Type11"] == 1, "P_Cover_Type_2"] = 0.2
test_forest_3.loc[test_forest_3["Soil_Type11"] == 1, "P_Cover_Type_3"] = 0.23
test_forest_3.loc[test_forest_3["Soil_Type11"] == 1, "P_Cover_Type_5"] = 0.46
test_forest_3.loc[test_forest_3["Soil_Type11"] == 1, "P_Cover_Type_6"] = 0.1
test_forest_3.loc[test_forest_3["Soil_Type13"] == 1, "P_Cover_Type_1"] = 0.03
test_forest_3.loc[test_forest_3["Soil_Type13"] == 1, "P_Cover_Type_2"] = 0.18
test_forest_3.loc[test_forest_3["Soil_Type13"] == 1, "P_Cover_Type_3"] = 0.01
test_forest_3.loc[test_forest_3["Soil_Type13"] == 1, "P_Cover_Type_5"] = 0.64
test_forest_3.loc[test_forest_3["Soil_Type13"] == 1, "P_Cover_Type_6"] = 0.14
test_forest_3.loc[test_forest_3["Soil_Type14"] == 1, "P_Cover_Type_3"] = 0.06
test_forest_3.loc[test_forest_3["Soil_Type14"] == 1, "P_Cover_Type_6"] = 0.94
test_forest_3.loc[test_forest_3["Soil_Type16"] == 1, "P_Cover_Type_2"] = 0.07
test_forest_3.loc[test_forest_3["Soil_Type16"] == 1, "P_Cover_Type_3"] = 0.07
test_forest_3.loc[test_forest_3["Soil_Type16"] == 1, "P_Cover_Type_6"] = 0.86
test_forest_3.loc[test_forest_3["Soil_Type17"] == 1, "P_Cover_Type_1"] = 0.01
test_forest_3.loc[test_forest_3["Soil_Type17"] == 1, "P_Cover_Type_2"] = 0.03
test_forest_3.loc[test_forest_3["Soil_Type17"] == 1, "P_Cover_Type_3"] = 0.05
test_forest_3.loc[test_forest_3["Soil_Type17"] == 1, "P_Cover_Type_5"] = 0.57
test_forest_3.loc[test_forest_3["Soil_Type17"] == 1, "P_Cover_Type_6"] = 0.34
test_forest_3.loc[test_forest_3["Soil_Type19"] == 1, "P_Cover_Type_1"] = 0.14
test_forest_3.loc[test_forest_3["Soil_Type19"] == 1, "P_Cover_Type_2"] = 0.05
test_forest_3.loc[test_forest_3["Soil_Type19"] == 1, "P_Cover_Type_5"] = 0.81
test_forest_3.loc[test_forest_3["Soil_Type20"] == 1, "P_Cover_Type_1"] = 0.21
test_forest_3.loc[test_forest_3["Soil_Type20"] == 1, "P_Cover_Type_2"] = 0.1
test_forest_3.loc[test_forest_3["Soil_Type20"] == 1, "P_Cover_Type_5"] = 0.1
test_forest_3.loc[test_forest_3["Soil_Type20"] == 1, "P_Cover_Type_6"] = 0.59
test_forest_3.loc[test_forest_3["Soil_Type21"] == 1, "P_Cover_Type_1"] = 0.75
test_forest_3.loc[test_forest_3["Soil_Type21"] == 1, "P_Cover_Type_7"] = 0.25
test_forest_3.loc[test_forest_3["Soil_Type22"] == 1, "P_Cover_Type_1"] = 0.76
test_forest_3.loc[test_forest_3["Soil_Type22"] == 1, "P_Cover_Type_2"] = 0.07
test_forest_3.loc[test_forest_3["Soil_Type22"] == 1, "P_Cover_Type_7"] = 0.17
test_forest_3.loc[test_forest_3["Soil_Type23"] == 1, "P_Cover_Type_1"] = 0.41
test_forest_3.loc[test_forest_3["Soil_Type23"] == 1, "P_Cover_Type_2"] = 0.17
test_forest_3.loc[test_forest_3["Soil_Type23"] == 1, "P_Cover_Type_5"] = 0.29
test_forest_3.loc[test_forest_3["Soil_Type23"] == 1, "P_Cover_Type_6"] = 0.01
test_forest_3.loc[test_forest_3["Soil_Type23"] == 1, "P_Cover_Type_7"] = 0.12
test_forest_3.loc[test_forest_3["Soil_Type24"] == 1, "P_Cover_Type_1"] = 0.61
test_forest_3.loc[test_forest_3["Soil_Type24"] == 1, "P_Cover_Type_2"] = 0.24
test_forest_3.loc[test_forest_3["Soil_Type24"] == 1, "P_Cover_Type_6"] = 0.08
test_forest_3.loc[test_forest_3["Soil_Type24"] == 1, "P_Cover_Type_7"] = 0.07
test_forest_3.loc[test_forest_3["Soil_Type26"] == 1, "P_Cover_Type_1"] = 0.13
test_forest_3.loc[test_forest_3["Soil_Type26"] == 1, "P_Cover_Type_2"] = 0.35
test_forest_3.loc[test_forest_3["Soil_Type26"] == 1, "P_Cover_Type_5"] = 0.52
test_forest_3.loc[test_forest_3["Soil_Type27"] == 1, "P_Cover_Type_1"] = 0.47
test_forest_3.loc[test_forest_3["Soil_Type27"] == 1, "P_Cover_Type_2"] = 0.33
test_forest_3.loc[test_forest_3["Soil_Type27"] == 1, "P_Cover_Type_7"] = 0.2
test_forest_3.loc[test_forest_3["Soil_Type28"] == 1, "P_Cover_Type_2"] = 0.67
test_forest_3.loc[test_forest_3["Soil_Type28"] == 1, "P_Cover_Type_7"] = 0.33
test_forest_3.loc[test_forest_3["Soil_Type31"] == 1, "P_Cover_Type_1"] = 0.33
test_forest_3.loc[test_forest_3["Soil_Type31"] == 1, "P_Cover_Type_2"] = 0.3
test_forest_3.loc[test_forest_3["Soil_Type31"] == 1, "P_Cover_Type_5"] = 0.27
test_forest_3.loc[test_forest_3["Soil_Type31"] == 1, "P_Cover_Type_6"] = 0.02
test_forest_3.loc[test_forest_3["Soil_Type31"] == 1, "P_Cover_Type_7"] = 0.08
test_forest_3.loc[test_forest_3["Soil_Type32"] == 1, "P_Cover_Type_1"] = 0.32
test_forest_3.loc[test_forest_3["Soil_Type32"] == 1, "P_Cover_Type_2"] = 0.37
test_forest_3.loc[test_forest_3["Soil_Type32"] == 1, "P_Cover_Type_3"] = 0.01
test_forest_3.loc[test_forest_3["Soil_Type32"] == 1, "P_Cover_Type_5"] = 0.15
test_forest_3.loc[test_forest_3["Soil_Type32"] == 1, "P_Cover_Type_6"] = 0.05
test_forest_3.loc[test_forest_3["Soil_Type32"] == 1, "P_Cover_Type_7"] = 0.1
test_forest_3.loc[test_forest_3["Soil_Type33"] == 1, "P_Cover_Type_1"] = 0.27
test_forest_3.loc[test_forest_3["Soil_Type33"] == 1, "P_Cover_Type_2"] = 0.31
test_forest_3.loc[test_forest_3["Soil_Type33"] == 1, "P_Cover_Type_5"] = 0.2
test_forest_3.loc[test_forest_3["Soil_Type33"] == 1, "P_Cover_Type_6"] = 0.11
test_forest_3.loc[test_forest_3["Soil_Type33"] == 1, "P_Cover_Type_7"] = 0.11
test_forest_3.loc[test_forest_3["Soil_Type34"] == 1, "P_Cover_Type_2"] = 0.55
test_forest_3.loc[test_forest_3["Soil_Type34"] == 1, "P_Cover_Type_5"] = 0.18
test_forest_3.loc[test_forest_3["Soil_Type34"] == 1, "P_Cover_Type_6"] = 0.09
test_forest_3.loc[test_forest_3["Soil_Type34"] == 1, "P_Cover_Type_7"] = 0.18
test_forest_3.loc[test_forest_3["Soil_Type35"] == 1, "P_Cover_Type_7"] = 1
test_forest_3.loc[test_forest_3["Soil_Type36"] == 1, "P_Cover_Type_2"] = 0.2
test_forest_3.loc[test_forest_3["Soil_Type36"] == 1, "P_Cover_Type_7"] = 0.8
test_forest_3.loc[test_forest_3["Soil_Type37"] == 1, "P_Cover_Type_7"] = 1
test_forest_3.loc[test_forest_3["Soil_Type38"] == 1, "P_Cover_Type_1"] = 0.05
test_forest_3.loc[test_forest_3["Soil_Type38"] == 1, "P_Cover_Type_7"] = 0.95
test_forest_3.loc[test_forest_3["Soil_Type39"] == 1, "P_Cover_Type_1"] = 0.04
test_forest_3.loc[test_forest_3["Soil_Type39"] == 1, "P_Cover_Type_2"] = 0.01
test_forest_3.loc[test_forest_3["Soil_Type39"] == 1, "P_Cover_Type_7"] = 0.95
test_forest_3.loc[test_forest_3["Soil_Type40"] == 1, "P_Cover_Type_1"] = 0.01
test_forest_3.loc[test_forest_3["Soil_Type40"] == 1, "P_Cover_Type_2"] = 0.01
test_forest_3.loc[test_forest_3["Soil_Type40"] == 1, "P_Cover_Type_7"] = 0.98

forest_4.loc[forest_4["Soil_Type1"] == 1, "P_Cover_Type_3"] = 0.34
forest_4.loc[forest_4["Soil_Type1"] == 1, "P_Cover_Type_4"] = 0.39
forest_4.loc[forest_4["Soil_Type1"] == 1, "P_Cover_Type_6"] = 0.27
forest_4.loc[forest_4["Soil_Type2"] == 1, "P_Cover_Type_3"] = 0.31
forest_4.loc[forest_4["Soil_Type2"] == 1, "P_Cover_Type_4"] = 0.33
forest_4.loc[forest_4["Soil_Type2"] == 1, "P_Cover_Type_6"] = 0.36
forest_4.loc[forest_4["Soil_Type3"] == 1, "P_Cover_Type_3"] = 0.09
forest_4.loc[forest_4["Soil_Type3"] == 1, "P_Cover_Type_4"] = 0.9
forest_4.loc[forest_4["Soil_Type3"] == 1, "P_Cover_Type_6"] = 0.01
forest_4.loc[forest_4["Soil_Type4"] == 1, "P_Cover_Type_3"] = 0.25
forest_4.loc[forest_4["Soil_Type4"] == 1, "P_Cover_Type_4"] = 0.69
forest_4.loc[forest_4["Soil_Type4"] == 1, "P_Cover_Type_6"] = 0.06
forest_4.loc[forest_4["Soil_Type5"] == 1, "P_Cover_Type_3"] = 0.33
forest_4.loc[forest_4["Soil_Type5"] == 1, "P_Cover_Type_4"] = 0.24
forest_4.loc[forest_4["Soil_Type5"] == 1, "P_Cover_Type_6"] = 0.43
forest_4.loc[forest_4["Soil_Type6"] == 1, "P_Cover_Type_2"] = 0.01
forest_4.loc[forest_4["Soil_Type6"] == 1, "P_Cover_Type_3"] = 0.38
forest_4.loc[forest_4["Soil_Type6"] == 1, "P_Cover_Type_4"] = 0.38
forest_4.loc[forest_4["Soil_Type6"] == 1, "P_Cover_Type_6"] = 0.23
forest_4.loc[forest_4["Soil_Type10"] == 1, "P_Cover_Type_2"] = 0.01
forest_4.loc[forest_4["Soil_Type10"] == 1, "P_Cover_Type_3"] = 0.42
forest_4.loc[forest_4["Soil_Type10"] == 1, "P_Cover_Type_4"] = 0.11
forest_4.loc[forest_4["Soil_Type10"] == 1, "P_Cover_Type_6"] = 0.46
forest_4.loc[forest_4["Soil_Type11"] == 1, "P_Cover_Type_3"] = 0.19
forest_4.loc[forest_4["Soil_Type11"] == 1, "P_Cover_Type_4"] = 0.33
forest_4.loc[forest_4["Soil_Type11"] == 1, "P_Cover_Type_6"] = 0.48
forest_4.loc[forest_4["Soil_Type14"] == 1, "P_Cover_Type_3"] = 0.02
forest_4.loc[forest_4["Soil_Type14"] == 1, "P_Cover_Type_4"] = 0.84
forest_4.loc[forest_4["Soil_Type14"] == 1, "P_Cover_Type_6"] = 0.14
forest_4.loc[forest_4["Soil_Type15"] == 1, "P_Cover_Type_4"] = 1
forest_4.loc[forest_4["Soil_Type16"] == 1, "P_Cover_Type_3"] = 0.05
forest_4.loc[forest_4["Soil_Type16"] == 1, "P_Cover_Type_4"] = 0.71
forest_4.loc[forest_4["Soil_Type16"] == 1, "P_Cover_Type_6"] = 0.24
forest_4.loc[forest_4["Soil_Type17"] == 1, "P_Cover_Type_3"] = 0.06
forest_4.loc[forest_4["Soil_Type17"] == 1, "P_Cover_Type_4"] = 0.92
forest_4.loc[forest_4["Soil_Type17"] == 1, "P_Cover_Type_6"] = 0.02

test_forest_4.loc[test_forest_4["Soil_Type1"] == 1, "P_Cover_Type_3"] = 0.34
test_forest_4.loc[test_forest_4["Soil_Type1"] == 1, "P_Cover_Type_4"] = 0.39
test_forest_4.loc[test_forest_4["Soil_Type1"] == 1, "P_Cover_Type_6"] = 0.27
test_forest_4.loc[test_forest_4["Soil_Type2"] == 1, "P_Cover_Type_3"] = 0.31
test_forest_4.loc[test_forest_4["Soil_Type2"] == 1, "P_Cover_Type_4"] = 0.33
test_forest_4.loc[test_forest_4["Soil_Type2"] == 1, "P_Cover_Type_6"] = 0.36
test_forest_4.loc[test_forest_4["Soil_Type3"] == 1, "P_Cover_Type_3"] = 0.09
test_forest_4.loc[test_forest_4["Soil_Type3"] == 1, "P_Cover_Type_4"] = 0.9
test_forest_4.loc[test_forest_4["Soil_Type3"] == 1, "P_Cover_Type_6"] = 0.01
test_forest_4.loc[test_forest_4["Soil_Type4"] == 1, "P_Cover_Type_3"] = 0.25
test_forest_4.loc[test_forest_4["Soil_Type4"] == 1, "P_Cover_Type_4"] = 0.69
test_forest_4.loc[test_forest_4["Soil_Type4"] == 1, "P_Cover_Type_6"] = 0.06
test_forest_4.loc[test_forest_4["Soil_Type5"] == 1, "P_Cover_Type_3"] = 0.33
test_forest_4.loc[test_forest_4["Soil_Type5"] == 1, "P_Cover_Type_4"] = 0.24
test_forest_4.loc[test_forest_4["Soil_Type5"] == 1, "P_Cover_Type_6"] = 0.43
test_forest_4.loc[test_forest_4["Soil_Type6"] == 1, "P_Cover_Type_2"] = 0.01
test_forest_4.loc[test_forest_4["Soil_Type6"] == 1, "P_Cover_Type_3"] = 0.38
test_forest_4.loc[test_forest_4["Soil_Type6"] == 1, "P_Cover_Type_4"] = 0.38
test_forest_4.loc[test_forest_4["Soil_Type6"] == 1, "P_Cover_Type_6"] = 0.23
test_forest_4.loc[test_forest_4["Soil_Type10"] == 1, "P_Cover_Type_2"] = 0.01
test_forest_4.loc[test_forest_4["Soil_Type10"] == 1, "P_Cover_Type_3"] = 0.42
test_forest_4.loc[test_forest_4["Soil_Type10"] == 1, "P_Cover_Type_4"] = 0.11
test_forest_4.loc[test_forest_4["Soil_Type10"] == 1, "P_Cover_Type_6"] = 0.46
test_forest_4.loc[test_forest_4["Soil_Type11"] == 1, "P_Cover_Type_3"] = 0.19
test_forest_4.loc[test_forest_4["Soil_Type11"] == 1, "P_Cover_Type_4"] = 0.33
test_forest_4.loc[test_forest_4["Soil_Type11"] == 1, "P_Cover_Type_6"] = 0.48
test_forest_4.loc[test_forest_4["Soil_Type14"] == 1, "P_Cover_Type_3"] = 0.02
test_forest_4.loc[test_forest_4["Soil_Type14"] == 1, "P_Cover_Type_4"] = 0.84
test_forest_4.loc[test_forest_4["Soil_Type14"] == 1, "P_Cover_Type_6"] = 0.14
test_forest_4.loc[test_forest_4["Soil_Type15"] == 1, "P_Cover_Type_4"] = 1
test_forest_4.loc[test_forest_4["Soil_Type16"] == 1, "P_Cover_Type_3"] = 0.05
test_forest_4.loc[test_forest_4["Soil_Type16"] == 1, "P_Cover_Type_4"] = 0.71
test_forest_4.loc[test_forest_4["Soil_Type16"] == 1, "P_Cover_Type_6"] = 0.24
test_forest_4.loc[test_forest_4["Soil_Type17"] == 1, "P_Cover_Type_3"] = 0.06
test_forest_4.loc[test_forest_4["Soil_Type17"] == 1, "P_Cover_Type_4"] = 0.92
test_forest_4.loc[test_forest_4["Soil_Type17"] == 1, "P_Cover_Type_6"] = 0.02


In [17]:
forest_3.describe()

,Elevation,Aspect,Slope,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Hillshade_3pm,Horizontal_Distance_To_Fire_Points,...,Soil_Type37,Soil_Type38,Soil_Type39,Soil_Type40,P_Cover_Type_1,P_Cover_Type_2,P_Cover_Type_3,P_Cover_Type_5,P_Cover_Type_6,P_Cover_Type_7
count,6349.000000,6349.000000,6349.000000,6349.000000,6349.000000,6349.000000,6349.000000,6349.000000,6349.000000,6349.000000,...,6349.000000,6349.000000,6349.000000,6349.000000,6349.000000,6349.000000,6349.000000,6349.000000,6349.000000,6349.000000
mean,2923.116081,160.891794,15.378485,276.708458,56.106473,1907.727359,212.103166,224.020318,142.182706,1518.206017,...,0.000945,0.068672,0.068200,0.033864,0.142986,0.149072,0.136226,0.205183,0.150794,0.214698
std,319.124056,108.490632,7.431408,246.691628,66.586037,1072.201316,26.518839,20.681232,39.944109,842.253981,...,0.030729,0.252916,0.252108,0.180892,0.182034,0.133107,0.227905,0.193104,0.199797,0.356323
min,2301.000000,0.000000,0.000000,0.000000,-123.000000,0.000000,0.000000,99.000000,0.000000,30.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,2655.000000,64.000000,10.000000,90.000000,7.000000,1129.000000,196.000000,212.000000,118.000000,870.000000,...,0.000000,0.000000,0.000000,0.000000,0.010000,0.030000,0.000000,0.000000,0.010000,0.000000
50%,2869.000000,149.000000,14.000000,212.000000,37.000000,1776.000000,217.000000,226.000000,143.000000,1368.000000,...,0.000000,0.000000,0.000000,0.000000,0.040000,0.100000,0.010000,0.180000,0.100000,0.080000
75%,3226.000000,252.000000,20.000000,390.000000,84.000000,2460.000000,231.000000,241.000000,171.000000,2018.000000,...,0.000000,0.000000,0.000000,0.000000,0.320000,0.300000,0.150000,0.270000,0.140000,0.120000
max,3849.000000,359.000000,52.000000,1343.000000,554.000000,5463.000000,254.000000,254.000000,248.000000,4481.000000,...,1.000000,1.000000,1.000000,1.000000,0.760000,0.670000,0.700000,0.810000,0.940000,1.000000


# Features removing

In [18]:
# will mask
#test_forest_2.drop(["Soil_Type16", "Soil_Type18"], axis=1, inplace = True)

In [19]:
#forest_1.drop(["Soil_Type8"], axis=1, inplace = True)# forest_1.drop(["Soil_Type8"], axis=1, inplace = True)
forest_2.drop(["Soil_Type35"], axis=1, inplace = True)#forest_2.drop(["Soil_Type24", "Soil_Type35"], axis=1, inplace = True)
forest_3.drop(["Aspect", "Slope", "Soil_Type19"], axis=1, inplace = True) #,"Hillshade_3pm", "Soil_Type19" forest_3.drop(["Aspect", "Slope"], axis=1, inplace = True) #, "Soil_Type19"
forest_4.drop(["Soil_Type14"], axis=1, inplace = True)
test_forest_2.drop(["Soil_Type35"], axis=1, inplace = True)
test_forest_3.drop(["Aspect", "Slope", "Soil_Type19"], axis=1, inplace = True)
test_forest_4.drop(["Soil_Type14"], axis=1, inplace = True)

##forest_1.drop(["Soil_Type8", "Soil_Type37",    "Soil_Type39"], axis=1, inplace = True)# forest_1.drop(["Soil_Type8"], axis=1, inplace = True)
##forest_2.drop(["Soil_Type13", "Soil_Type19", "Soil_Type20", "Soil_Type25", "Soil_Type29", "Soil_Type31", "Soil_Type39"], axis=1, inplace = True)#forest_2.drop(["Soil_Type24", "Soil_Type35"], axis=1, inplace = True)
##forest_3.drop(["Soil_Type14", "Soil_Type37", "Slope",  "Aspect"], axis=1, inplace = True) #,"Hillshade_3pm", "Soil_Type19" forest_3.drop(["Aspect", "Slope"], axis=1, inplace = True) #, "Soil_Type19"
#forest_4.drop(["Slope"], axis=1, inplace = True)#"Slope",forest_4.drop(["Soil_Type14"], axis=1, inplace = True)#"Slope",

# Models

## KNN

In [20]:
y_test_all = pd.DataFrame()
y_test_pred_all = pd.DataFrame()

for ind in range(0,4):
    print("Forest", ind + 1)
    X_train, X_valid, y_train, y_valid = train_test_split(forest_list[ind], y_forest_list[ind], test_size=0.2, random_state=42)
    scaler = MinMaxScaler()
    X_train_sc = pd.DataFrame(scaler.fit_transform(X_train), columns=X_train.columns)
    X_valid_sc = pd.DataFrame(scaler.transform(X_valid), columns=X_valid.columns)
    clf = KNeighborsClassifier(n_neighbors=1)
    clf.fit(X_train_sc, y_train)
    y_pred = clf.predict(X_valid_sc)
    print("\taccuracy: {:.4f}".format(accuracy_score(y_valid, y_pred)))
    y_test_all = pd.concat([y_test_all, y_valid], axis=0, ignore_index=True)
    y_test_pred_all = pd.concat([y_test_pred_all, pd.DataFrame(y_pred, columns=['Cover_Type'])], axis=0, ignore_index=True)
print("summ accuracy: {:.4f}".format(accuracy_score(y_test_all, y_test_pred_all)))

Forest 1
	accuracy: 0.8306
Forest 2
	accuracy: 0.8317
Forest 3
	accuracy: 0.8472
Forest 4
	accuracy: 0.8825
summ accuracy: 0.8537


In [21]:
"""
res_all = pd.DataFrame()

for ind in range(0, 4):
    forest_list[ind] = forest_list[ind].reset_index(drop=True)
    y_forest_list[ind] = y_forest_list[ind].reset_index(drop=True)
    test_list[ind] = test_list[ind].reset_index(drop=True)

    scaler = MinMaxScaler()  #RobustScaler()  StandardScaler()
    fr_sc = pd.DataFrame(scaler.fit_transform(forest_list[ind]), columns=forest_list[ind].columns)
    test_sc = pd.DataFrame(scaler.transform(test_list[ind]), columns=test_list[ind].columns)

    clf = KNeighborsClassifier(n_neighbors=1)
    clf.fit(fr_sc, y_forest_list[ind])
    pred = clf.predict(test_sc)

    res = res_ind_list[ind].copy()
    res = pd.concat([res, pd.DataFrame(pred, columns=['Cover_Type'])], axis=1)
    res_all = pd.concat([res_all, res], axis=0)
    res_all = res_all.reset_index(drop=True)

res_all.sort_values(by="Id", inplace=True)
res_all.to_csv("ForestsVIV_5.csv", sep=",", index=False)
"""

'\nres_all = pd.DataFrame()\n\nfor ind in range(0, 4):\n    forest_list[ind] = forest_list[ind].reset_index(drop=True)\n    y_forest_list[ind] = y_forest_list[ind].reset_index(drop=True)\n    test_list[ind] = test_list[ind].reset_index(drop=True)\n\n    scaler = MinMaxScaler()  #RobustScaler()  StandardScaler()\n    fr_sc = pd.DataFrame(scaler.fit_transform(forest_list[ind]), columns=forest_list[ind].columns)\n    test_sc = pd.DataFrame(scaler.transform(test_list[ind]), columns=test_list[ind].columns)\n\n    clf = KNeighborsClassifier(n_neighbors=1)\n    clf.fit(fr_sc, y_forest_list[ind])\n    pred = clf.predict(test_sc)\n\n    res = res_ind_list[ind].copy()\n    res = pd.concat([res, pd.DataFrame(pred, columns=[\'Cover_Type\'])], axis=1)\n    res_all = pd.concat([res_all, res], axis=0)\n    res_all = res_all.reset_index(drop=True)\n\nres_all.sort_values(by="Id", inplace=True)\nres_all.to_csv("ForestsVIV_5.csv", sep=",", index=False)\n'

## XGBClassifier

In [22]:
"""
y_test_all = pd.DataFrame()
y_test_pred_all = pd.DataFrame()
n_estimators_list = [250, 50, 700, 600]
max_depth_list = [7, 5, 6, 6]

for ind in range(0,4):
    print("Forest", ind + 1)
    X_train, X_valid, y_train, y_valid = train_test_split(forest_list[ind], y_forest_list[ind], test_size=0.2, random_state=42)
    model_xgb = XGBClassifier(silent=False, nthread=4, max_depth=max_depth_list[ind], n_estimators=n_estimators_list[ind], random_state=42, scoring="accuracy") #  subsample=0.5, learning_rate=0.03,
    model_xgb.fit(X_train, y_train)
    y_pred = model_xgb.predict(X_valid)
    print("\taccuracy: {:.4f}".format(accuracy_score(y_valid, y_pred)))
    y_test_all = pd.concat([y_test_all, y_valid], axis=0, ignore_index=True)
    y_test_pred_all = pd.concat([y_test_pred_all, pd.DataFrame(y_pred, columns=['Cover_Type'])], axis=0, ignore_index=True)
    print(model_xgb.feature_importances_)
print("summ accuracy: {:.4f}".format(accuracy_score(y_test_all, y_test_pred_all)))
"""

'\ny_test_all = pd.DataFrame()\ny_test_pred_all = pd.DataFrame()\nn_estimators_list = [250, 50, 700, 600]\nmax_depth_list = [7, 5, 6, 6]\n\nfor ind in range(0,4):\n    print("Forest", ind + 1)\n    X_train, X_valid, y_train, y_valid = train_test_split(forest_list[ind], y_forest_list[ind], test_size=0.2, random_state=42)\n    model_xgb = XGBClassifier(silent=False, nthread=4, max_depth=max_depth_list[ind], n_estimators=n_estimators_list[ind], random_state=42, scoring="accuracy") #  subsample=0.5, learning_rate=0.03,\n    model_xgb.fit(X_train, y_train)\n    y_pred = model_xgb.predict(X_valid)\n    print("\taccuracy: {:.4f}".format(accuracy_score(y_valid, y_pred)))\n    y_test_all = pd.concat([y_test_all, y_valid], axis=0, ignore_index=True)\n    y_test_pred_all = pd.concat([y_test_pred_all, pd.DataFrame(y_pred, columns=[\'Cover_Type\'])], axis=0, ignore_index=True)\n    print(model_xgb.feature_importances_)\nprint("summ accuracy: {:.4f}".format(accuracy_score(y_test_all, y_test_pred_all

In [23]:
res_all = pd.DataFrame()
re_1 = pd.DataFrame()
re_2 = pd.DataFrame()
re_3 = pd.DataFrame()
re_4 = pd.DataFrame()
re_list = [re_1, re_2, re_3, re_4]
n_estimators_list = [250, 50, 700, 600]
max_depth_list = [8, 6, 8, 7]

for ind in range(0, 4):
    print(ind+1)
    forest_list[ind] = forest_list[ind].reset_index(drop=True)
    y_forest_list[ind] = y_forest_list[ind].reset_index(drop=True)
    test_list[ind] = test_list[ind].reset_index(drop=True)

    model_xgb = XGBClassifier(silent=False, nthread=4, max_depth=max_depth_list[ind], n_estimators=n_estimators_list[ind], random_state=42, scoring="accuracy")
    model_xgb.fit(forest_list[ind], y_forest_list[ind])
    pred = model_xgb.predict(test_list[ind])

    res = res_ind_list[ind].copy()
    res = pd.concat([res, pd.DataFrame(pred, columns=['Cover_Type'])], axis=1)
    re_list[ind] = res.copy()
    res_all = pd.concat([res_all, res], axis=0)
    res_all = res_all.reset_index(drop=True)

res_all.sort_values(by="Id", inplace=True)
res_all.to_csv("ForestsVIV_11.csv", sep=",", index=False)
res_all.info()

1
2
3
4
<class 'pandas.core.frame.DataFrame'>
Int64Index: 565892 entries, 0 to 533598
Data columns (total 2 columns):
Id            565892 non-null int64
Cover_Type    565892 non-null int64
dtypes: int64(2)
memory usage: 13.0 MB
